In [1]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core.databricks import PyPiLibrary

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.train.hyperdrive import choice, loguniform

from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.47.0


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


learningmain
learning
eastus
dac8073e-1c2d-4a7d-a53b-c3655e291d58


In [28]:
from azureml.core import Environment

jdk_env = Environment("jdk_test")


In [29]:
%%writefile conda.yaml

channels:
  - pytorch
  - anaconda
  - conda-forge
dependencies:
  - python=3.7
  - pip=21.1.2
  - pip:
      - azure-ai-ml==1.2.0
      - mlflow== 1.26.1
      - azureml-mlflow==1.42.0
      - nvitop
      - transformers
      - inference-schema
      - joblib
      - datasets
      - sentencepiece
  - numpy~=1.21.6
  - pandas~=1.1.5
  - shap=0.39.0
  - scikit-learn~=0.22.1
  - pytorch==1.7.1
name: nlp_training_environment


Overwriting conda.yaml


In [30]:
# jdk_env = Environment.from_conda_specification('nlp_training_environment', 'conda.yaml')


In [31]:
docker_file="""
FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.3-cudnn8-ubuntu20.04

WORKDIR /

ENV CONDA_PREFIX=/azureml-envs/pytorch-1.11
ENV CONDA_DEFAULT_ENV=$CONDA_PREFIX
ENV PATH=$CONDA_PREFIX/bin:$PATH

# This is needed for mpi to locate libpython
ENV LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH

# Create conda environment
COPY conda_dependencies.yaml .
RUN conda env create -p $CONDA_PREFIX -f conda_dependencies.yaml -q && \
    rm conda_dependencies.yaml && \
    conda run -p $CONDA_PREFIX pip cache purge && \
    conda clean -a -y

"""

In [32]:
# Set the base image to None, because the image is defined by Dockerfile.
# jdk_env.docker.base_image = None
# jdk_env.docker.base_dockerfile = docker_file

In [33]:
docker_file = """
FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.3-cudnn8-ubuntu20.04

RUN mkdir -p /usr/share/man/man1 /usr/share/man/man2 && \
    apt-get update &&\
    apt-get install -y --no-install-recommends openjdk-11-jre && \
    apt-get install ca-certificates-java -y && \
    apt-get clean && \
    update-ca-certificates -f;
ENV JAVA_HOME /usr/lib/jvm/java-11-openjdk-amd64/

ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/responsibleai-0.20

# Install wkhtmltopdf for pdf rendering from html
RUN apt-get -y update && apt-get -y install wkhtmltopdf

# Create conda environment
RUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \
    python=3.8 pip=21.3.1 -c anaconda -c conda-forge

# Prepend path to AzureML conda environment
ENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH

# Install pip dependencies
# markupsafe and itsdangerous are bug workarounds
RUN pip install 'responsibleai~=0.20.0' \
                'mlflow' \
                'scikit-learn<1.1'

# This is needed for mpi to locate libpython
ENV LD_LIBRARY_PATH $AZUREML_CONDA_ENVIRONMENT_PATH/lib:$LD_LIBRARY_PATH


"""

In [ ]:
docker_file = """
FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.3-cudnn8-ubuntu20.04

RUN mkdir -p /usr/share/man/man1 /usr/share/man/man2 && \
    apt-get update &&\
    apt-get install -y --no-install-recommends openjdk-11-jre && \
    apt-get install ca-certificates-java -y && \
    apt-get clean && \
    update-ca-certificates -f;
ENV JAVA_HOME /usr/lib/jvm/java-11-openjdk-amd64/

ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/jdk-11-env

# Install wkhtmltopdf for pdf rendering from html
RUN apt-get -y update && apt-get -y install wkhtmltopdf

# Create conda environment
RUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \
    python=3.8 pip=21.3.1 -c anaconda -c conda-forge

# Prepend path to AzureML conda environment
ENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH

# Install pip dependencies
# markupsafe and itsdangerous are bug workarounds
RUN pip install 'responsibleai~=0.20.0' \
                'mlflow' \
                'scikit-learn<1.1'

# This is needed for mpi to locate libpython
ENV LD_LIBRARY_PATH $AZUREML_CONDA_ENVIRONMENT_PATH/lib:$LD_LIBRARY_PATH


"""

In [34]:
# Set the base image to None, because the image is defined by Dockerfile.
jdk_env.docker.base_image = None
jdk_env.docker.base_dockerfile = docker_file
jdk_env.python.user_managed_dependencies = True

# Alternatively, load the string from a file.
# jdk_env.docker.base_image = None
# jdk_env.docker.base_dockerfile = "./Dockerfile"


In [35]:
jdk_env = jdk_env.register(ws)